In [156]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta

from numba import njit
import plotly.graph_objs as go
from vectorbt.portfolio.enums import Direction, SizeType
import talib



## data collection

In [157]:
symbols = [
    "LTC-USDT",   # Litecoin
    "TRX-USDT",   # TRON
    "AVAX-USDT",  # Avalanche
    "LINK-USDT",  # Chainlink
    "ATOM-USDT",  # Cosmos
    "XMR-USDT",   # Monero
    "ETC-USDT",   # Ethereum Classic
    "BCH-USDT",   # Bitcoin Cash
    "ALGO-USDT",  # Algorand
    "APT-USDT"    # Aptos
    "BTC-USDT",   # Bitcoin
    "SOL-USDT",   # Solana
]
timeframe = "1day"
start_time = "2021-01-08 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")


# Fetch data for each symbol and store in a dictionary

dataframes = {}
for symbol in symbols:
    try:
        fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
        df = fetcher.fetch_candles_as_df()
        df['symbol'] = symbol
        dataframes[symbol] = df
    except Exception as e:
        print(f"Error fetching data: {e}")

symbols = list(dataframes.keys())



INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 3, Candles: 1507
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 3, Candles: 1507
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 1452
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 3, Candles: 1507
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 3, Candles: 1507
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 1433
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spo

Error fetching data: KuCoin API error: {'msg': 'Unsupported trading pair.', 'code': '400100'}


INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 1300


In [158]:
symbols = list(dataframes.keys())
symbols

['LTC-USDT',
 'TRX-USDT',
 'AVAX-USDT',
 'LINK-USDT',
 'ATOM-USDT',
 'XMR-USDT',
 'ETC-USDT',
 'BCH-USDT',
 'ALGO-USDT',
 'SOL-USDT']

In [159]:

def momentum_strategy(close, high, low, atr_length, ema_length, ema2_length, vola_multiplier, vol_window, symbol=None):
    # Calculate EMA
    EMA = vbt.IndicatorFactory.from_talib('EMA')
    ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()
    ema2 = EMA.run(close, timeperiod=ema2_length).real.to_numpy()

    # Calculate ATR
    ATR = vbt.IndicatorFactory.from_talib('ATR')
    atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

    # Function to calculate rolling max
    def rolling_max(arr, window):
        result = np.full_like(arr, np.nan)
        for i in range(window - 1, len(arr)):
            result[i] = np.max(arr[i - window + 1:i + 1])
        return result

    # Function to compute bearish volatility
    def compute_is_bearish_vol(high, low, atr, window):
        rm = rolling_max(high, window=window)
        return (rm - low) > (atr * vola_multiplier)

    # Calculate conditions
    is_bullish = (close > ema) & (close > ema2)
    is_bearish_vol = compute_is_bearish_vol(high, low, atr, window=vol_window)
    is_caution = is_bullish & (is_bearish_vol | (close < ema))
    signal_buy = is_bullish & ~is_caution

    # Define conditions and corresponding values
    conditions = [signal_buy, is_caution]
    values = [10, 5]

    # Create final signal using np.select
    signal = np.select(conditions, values, default=0)

    return signal.reshape(close.shape)

# Create the indicator factory
momentum_indicator = vbt.IndicatorFactory(
    class_name='MomentumStrategy',
    short_name='momentum',
    input_names=['Close', 'High', 'Low'],
    param_names=['atr_length', 'ema_length', 'ema2_length', 'vola_multiplier', 'vol_window','symbol'],
    output_names=['signal']
).from_apply_func(momentum_strategy)


In [160]:

# # Define parameter ranges
# atr_length = np.arange(5, 8, 1)  # Range from 5 to 7 with step 1
# ema_length = np.arange(100, 241, 20)  # Range from 100 to 240 with step 20
# ema2_length = np.arange(10, 50, 5)  # Range from 10 to 45 with step 5
# vola_multiplier = [1.6, 1.7, 1.8]  # Specific values for volatility multiplier
# vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

# # Define parameter ranges
atr_length = 7# Range from 5 to 7 with step 1
ema_length = 200  # Range from 100 to 240 with step 20
ema2_length = 50  # Range from 10 to 45 with step 5
vola_multiplier = 1.5 # Specific values for volatility multiplier
vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

indicator_obj = {}

for symbol in symbols:
    # Run the indicator
    df_pair = dataframes[symbol]

    indicator = momentum_indicator.run(
        df_pair['close'], df_pair['high'], df_pair['low'],
        atr_length=atr_length,
        ema_length=ema_length,
        ema2_length=ema2_length,
        vola_multiplier=vola_multiplier,
        vol_window=vol_window,
        symbol= symbol,
        param_product=True
    )
    indicator_obj[symbol] = indicator

indicator_obj

{'LTC-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12d295d30>,
 'TRX-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12a7fa0a0>,
 'AVAX-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12df27550>,
 'LINK-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12d7dc610>,
 'ATOM-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12df19eb0>,
 'XMR-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12b066370>,
 'ETC-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12e050eb0>,
 'BCH-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12df15610>,
 'ALGO-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12df331f0>,
 'SOL-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12e09c460>}

In [161]:
# # Entry parameter 
# stretch = 0.5
# long_term_ma_len = [100,200]
# adx_len = [5,6]
# atr_len = 3

# # exit parameter
# short_term_ma_len = [20,30,40]


# indicator_obj = {}
# for symbol in symbols:
#     print(symbol)
#     df = dataframes[symbol]

#     # Usage example (commented out)
#     indicator = custom_strategy_indicator.run(
#         df['close'], df['high'], df['low'],
#         stretch=stretch,
#         long_term_ma_len=long_term_ma_len,
#         short_term_ma_len=short_term_ma_len,
#         adx_len=adx_len,
#         atr_len=atr_len,
#         symbol=symbol,
#         param_product=True

#     )

#     indicator_obj[symbol] = indicator

# indicator_obj




In [162]:
def copy_rename_close_series(df_original, df_indicator_signals):

    def repeat_series_horizontally(series, target_shape):
        return np.tile(series.values.reshape(-1, 1), target_shape[1])

    # Assuming df and df_indicator_signals are already defined
    nedded_copies = len(df_indicator_signals.columns)

    # Repeat the 'close' series horizontally to match the shape of df_indicator_signals
    df_dublicate_close = pd.DataFrame(repeat_series_horizontally(df_original['close'], df_indicator_signals.shape), index=df_original.index)

    # Rename columns if shapes match
    # if df_indicator_signals.shape == df_dublicate_close.shape:
    df_dublicate_close.columns = df_indicator_signals.columns
    #     print("Columns have been renamed")
    # else:
    #     print("The DataFrames do not have the same shape.")
    
    
    return df_dublicate_close



df_dublicate_close_pairs ={}
for symbol in symbols:
    print(symbol)
    df = dataframes[symbol]
    df_dublicate_close = copy_rename_close_series(df, indicator_obj[symbol].signal)
    print('New Schape',df_dublicate_close.shape)
    df_dublicate_close_pairs[symbol] = df_dublicate_close
    # df_dublicate_close_pairs.append(df_dublicate_close)

print(df_dublicate_close_pairs.keys())






LTC-USDT
New Schape (1507, 3)
TRX-USDT
New Schape (1507, 3)
AVAX-USDT
New Schape (1452, 3)
LINK-USDT
New Schape (1507, 3)
ATOM-USDT
New Schape (1507, 3)
XMR-USDT
New Schape (1433, 3)
ETC-USDT
New Schape (1507, 3)
BCH-USDT
New Schape (1507, 3)
ALGO-USDT
New Schape (1507, 3)
SOL-USDT
New Schape (1300, 3)
dict_keys(['LTC-USDT', 'TRX-USDT', 'AVAX-USDT', 'LINK-USDT', 'ATOM-USDT', 'XMR-USDT', 'ETC-USDT', 'BCH-USDT', 'ALGO-USDT', 'SOL-USDT'])


In [163]:
missing_data_pairs = []
for symbol in symbols:
    df_check = dataframes[symbol]
    # Create a complete date range based on the start and end of your DataFrame
    complete_date_range = pd.date_range(start=df_check.index.min(), end=df_check.index.max(), freq='1D')

    # Check for missing dates
    missing_dates = complete_date_range.difference(df_check.index)

    if not missing_dates.empty:
        # print('missing data in:',symbol)
        # print("Missing dates:")
        # print(missing_dates)
        missing_data_pairs.append(symbol)
    # else:
         # print("All dates are accounted for.")
    
missing_data_pairs


[]

In [164]:
df_dublicate_close_pairs['SOL-USDT']

momentum_atr_length              7                  
momentum_ema_length            200                  
momentum_ema2_length            50                  
momentum_vola_multiplier       1.5                  
momentum_vol_window              7        8        9
momentum_symbol           SOL-USDT SOL-USDT SOL-USDT
timestamp                                           
2021-08-04 00:00:00+00:00   35.799   35.799   35.799
2021-08-05 00:00:00+00:00   37.350   37.350   37.350
2021-08-06 00:00:00+00:00   39.535   39.535   39.535
2021-08-07 00:00:00+00:00   39.333   39.333   39.333
2021-08-08 00:00:00+00:00   37.715   37.715   37.715
...                            ...      ...      ...
2025-02-19 00:00:00+00:00  168.927  168.927  168.927
2025-02-20 00:00:00+00:00  176.049  176.049  176.049
2025-02-21 00:00:00+00:00  168.920  168.920  168.920
2025-02-22 00:00:00+00:00  172.078  172.078  172.078
2025-02-23 00:00:00+00:00  168.244  168.244  168.244

[1300 rows x 3 columns]

In [165]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,tp_hit,entry_price,atr_values):
    close_price = c.close[c.i, c.col]

    # ====================== Active Position ======================
    if (c.position_now > 0):

        # ------------------------- Exit Position -------------------------

        if open_[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close Entire position

                price=open_[c.i],  

                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)


            return value
        
        # ------------------------- Take profit ----------------------
        
        # # if (tp_prices[c.i] < high[c.i]) & (tp_hit[c.i] == False):
        # if (high[c.i] >= tp_prices[c.i]) and (not tp_hit[c.i]):

        #     tp_hit[:] = True

        #     value_sell = vbt.portfolio.nb.order_nb(
        #         size= -0.5,  # Close position

        #         price=tp_prices[c.i],  # take profit at tp price 

        #         size_type=SizeType.Percent,
        #         direction=Direction.LongOnly,

        #         fees=0.001,
        #         slippage=0.002)
            
        #     return value_sell
        
        
        # ------------------------- Trailing SL Update ----------------------

        # sl update for long position
        if (entries[c.i-1, c.col] == 5):  
            highest_low = np.max(low[c.i-7:c.i])
            update2 = highest_low - atr_values[c.i] * 0.2
            if update2 > sl_prices[c.i]:
                sl_prices[:]= update2

        #losen sl for less volatility 
        if (entries[c.i-1, c.col] == 10): 
            highest_low = np.max(low[c.i-7:c.i])
            update = highest_low - atr_values[c.i]
            if update > sl_prices[c.i]:
                sl_prices[:]= update



    
    # ====================== Entry Position ======================
  

    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i-1, c.col] == 10:

            # reset values to default 
            sl_prices[:] = np.nan
            entry_price[:] = open_[c.i]
            tp_hit[:] = False

            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size

                price=open_[c.i],  # Current closing price
                
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            # set intial stop loss
            sl_prices[:] = low[c.i] - atr_values[c.i] 

            # set take profit price
            atr_multiple = 1
            tp_prices[:] = entry_price[:] + (atr_values[c.i] * atr_multiple)

            return order

    
    return vbt.portfolio.enums.NoOrder


In [166]:
pf_pairs = {}

for symbol in symbols:
    # candle data
    df = dataframes[symbol]
    close = df_dublicate_close_pairs[symbol]
    open_ = df['open'].to_numpy().flatten()
    high = df['high'].to_numpy().flatten()
    low = df['low'].to_numpy().flatten()

    entries = indicator_obj[symbol].signal.values

    atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

    # Create an array to store SL prices
    sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
    tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
    entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array
    tp_hit = np.full(close.shape[0], False)  # Use a 1D array


    # Create portfolio
    pf = vbt.Portfolio.from_order_func(
        close,           # Price DataFrame
        order_func_nb,
        high,
        low,
        open_,
        entries,    # Order function
        sl_prices,
        tp_prices,
        tp_hit,
        entry_price,
        atr_values,  # Pass the SL prices array
        init_cash=500  # Initial cash balance
    )

    pf_pairs[symbol] = pf


In [167]:
indicator_obj['SOL-USDT'].signal.columns



MultiIndex([(7, 200, 50, 1.5, 7, 'SOL-USDT'),
            (7, 200, 50, 1.5, 8, 'SOL-USDT'),
            (7, 200, 50, 1.5, 9, 'SOL-USDT')],
           names=['momentum_atr_length', 'momentum_ema_length', 'momentum_ema2_length', 'momentum_vola_multiplier', 'momentum_vol_window', 'momentum_symbol'])

In [168]:
# Combine the total return and max drawdown for each symbol
first_glance = []

for symbol in symbols:
    # if symbol not in missing_data_pairs:
    total_return = pf_pairs[symbol].total_return()
    max_dd = pf_pairs[symbol].max_drawdown()
    sharpe_ratio = pf_pairs[symbol].sharpe_ratio()
    sortino_ratio = pf_pairs[symbol].sortino_ratio()
    beta = pf_pairs[symbol].beta()
    profit_factor = pf_pairs[symbol].trades.profit_factor()
    win_rate = pf_pairs[symbol].trades.win_rate()
    total_trades = pf_pairs[symbol].trades.count()
        
        # return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
        # return_and_maxdd = pd.concat([total_return, max_dd,total_trades,win_rate,profit_factor], axis=1)

    return_and_maxdd = pd.concat([total_return, max_dd,total_trades,sharpe_ratio,sortino_ratio,win_rate,beta,profit_factor], axis=1)


    first_glance.append(return_and_maxdd)

# Concatenate all DataFrames along the rows
combined_df = pd.concat(first_glance, axis=0)

combined_df

total_return  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
7                   200                 50                   1.5                      7                   LTC-USDT            -0.825213   
                                                                                      8                   LTC-USDT            -0.765030   
                                                                                      9                   LTC-USDT            -0.630336   
                                                                                      7                   TRX-USDT             0.684923   
                                                                                      8                   TRX-USDT             0.542228   
                                                                                      9                   TRX-USDT             0.704748   
                                                                                      7                   AVAX-USDT            0.260408   
                                                                                      8                   AVAX-USDT            0.661200   
                                                                                      9                   AVAX-USDT            0.672996   
                                                                                      7                   LINK-USDT           -0.218438   
                                                                                      8                   LINK-USDT           -0.189693   
                                                                                      9                   LINK-USDT           -0.206923   
                                                                                      7                   ATOM-USDT           -0.875116   
                                                                                      8                   ATOM-USDT           -0.915283   
                                                                                      9                   ATOM-USDT           -0.844581   
                                                                                      7                   XMR-USDT            -0.541568   
                                                                                      8                   XMR-USDT            -0.491468   
                                                                                      9                   XMR-USDT            -0.506056   
                                                                                      7                   ETC-USDT            -0.430282   
                                                                                      8                   ETC-USDT            -0.253586   
                                                                                      9                   ETC-USDT            -0.112776   
                                                                                      7                   BCH-USDT            -0.556574   
                                                                                      8                   BCH-USDT            -0.627241   
                                                                                      9                   BCH-USDT            -0.616163   
                                                                                      7                   ALGO-USDT            0.337420   
                                                                                      8                   ALGO-USDT            0.143446   
                                                                                      9                   ALGO-USDT            0.707818   
                                                                                      7     

In [169]:
index = 1

combined_df.sort_values(by='total_return', ascending=False).head(50)
combined_df
test_df = combined_df.copy()
test_df=test_df.reset_index()
test_df.iloc[index]
total_return_index = test_df.columns.get_loc('total_return')
total_return_index
name_of_setup_to_investigate = tuple(test_df.iloc[index, :total_return_index ])
name_of_setup_to_investigate

(7, 200, 50, 1.5, 8, 'LTC-USDT')

In [170]:
name_of_setup_to_investigate[-1]

'LTC-USDT'

In [171]:
# pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate[1:]].plot().show()
pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate].plot().show()

In [172]:
pf_pairs['SOL-USDT'][name_of_setup_to_investigate].orders.records_readable


KeyError: (7, 200, 50, 1.5, 8, 'LTC-USDT')